In [1]:
import os
import tensorflow as tf
from typing import Any, Dict, Text
from modules.merge_dataset import merge_dataset
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import pipeline, components
from absl import logging
from tfx import components
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import \
    LatestBlessedModelStrategy
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [2]:
merge_dataset("data/movies/movies.csv", "data/ratings/ratings.csv")

'Merge dataset success'

In [3]:
PIPELINE_NAME = "movie-recommender-pipeline"

# pipeline inputs
MOVIES_DATA_ROOT = "data/movies"
RATING_DATA_ROOT = "data/merge"
MOVIES_TRANSFORM_MODULE_FILE = "modules/movies_transform.py"
RATING_TRANSFORM_MODULE_FILE = "modules/rating_transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [4]:
context = InteractiveContext(pipeline_root=pipeline_root)

In [5]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(
            name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
    ])
)       

In [6]:
movies_example_gen = components.CsvExampleGen(
    input_base=MOVIES_DATA_ROOT,
    output_config=output
)
context.run(movies_example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 14
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
rating_example_gen = components.CsvExampleGen(
    input_base=RATING_DATA_ROOT,
    output_config=output
)
context.run(rating_example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 15
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [8]:
movies_statistics_gen = components.StatisticsGen(
    examples=movies_example_gen.outputs["examples"]
)
context.run(movies_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 16
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [9]:
rating_statistics_gen = components.StatisticsGen(
    examples=rating_example_gen.outputs["examples"]
)
context.run(rating_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 17
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [10]:
movies_schema_gen = components.SchemaGen(
    statistics=movies_statistics_gen.outputs["statistics"]
)
context.run(movies_schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 18
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
rating_schema_gen = components.SchemaGen(
    statistics=rating_statistics_gen.outputs["statistics"]
)
context.run(rating_schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 19
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [12]:
movies_example_validator = components.ExampleValidator(
    statistics=movies_statistics_gen.outputs["statistics"],
    schema=movies_schema_gen.outputs["schema"],
)
context.run(movies_example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 20
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [13]:
rating_example_validator = components.ExampleValidator(
    statistics=rating_statistics_gen.outputs["statistics"],
    schema=rating_schema_gen.outputs["schema"],
)
context.run(rating_example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 21
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [14]:
movies_transform = components.Transform(
    examples=movies_example_gen.outputs["examples"],
    schema=movies_schema_gen.outputs["schema"],
    module_file=os.path.abspath(MOVIES_TRANSFORM_MODULE_FILE)
)
context.run(movies_transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\22\.temp_path\tftransform_tmp\55cfd2997fe64062b56f11ff1569926f\assets


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\22\.temp_path\tftransform_tmp\55cfd2997fe64062b56f11ff1569926f\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 22
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [15]:
rating_transform = components.Transform(
    examples=rating_example_gen.outputs["examples"],
    schema=rating_schema_gen.outputs["schema"],
    module_file=os.path.abspath(RATING_TRANSFORM_MODULE_FILE)
)
context.run(rating_transform)

INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\23\.temp_path\tftransform_tmp\cd9fbc584ac94a2e85cbd0e6a78a41bf\assets


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\23\.temp_path\tftransform_tmp\cd9fbc584ac94a2e85cbd0e6a78a41bf\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 23
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [16]:
trainer = components.Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=rating_transform.outputs["transformed_examples"],
    transform_graph=rating_transform.outputs["transform_graph"],
    schema=rating_transform.outputs["post_transform_schema"],
    # hyperparameters=tuner.outputs["best_hyperparameters"],
    train_args=trainer_pb2.TrainArgs(
        splits=["train"],
        num_steps=500,
    ),
    eval_args=trainer_pb2.EvalArgs(
        splits=["eval"],
        num_steps=100,
    ),
    custom_config={
        "epochs": 5,
        "movies": movies_transform.outputs["transformed_examples"],
        "movies_schema": movies_transform.outputs["post_transform_schema"],
    }
)
context.run(trainer)


Epoch 1/5


ERROR:absl:ERROR IN run_fn during fit:
Graph execution error:

Detected at node 'sequential/Cast' defined at (most recent call last):
    File "c:\Users\Zis\anaconda3\envs\movies-rec\lib\runpy.py", line 192, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Zis\anaconda3\envs\movies-rec\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\Zis\anaconda3\envs\movies-rec\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Zis\anaconda3\envs\movies-rec\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "c:\Users\Zis\anaconda3\envs\movies-rec\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Zis\anaconda3\envs\movies-rec\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Z

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
c:\Users\Zis\anaconda3\envs\movies-rec\lib\site-packages\keras\engine\functional.py:566: UserWarning: Input dict contained keys ['title_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
c:\Users\Zis\anaconda3\envs\movies-rec\lib\site-packages\keras\engine\functional.py:566: UserWarning: Input dict contained keys ['title_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
c:\Users\Zis\anaconda3\envs\movies-rec\lib\site-packages\keras\engine\functional.py:566: UserWarning: Input dict contained keys ['title_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Trainer\model\24\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Trainer\model\24\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 24
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [17]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolve")
context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolve
    execution_id: 25
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolve, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolve, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [18]:
pusher = components.Pusher(
    model=trainer.outputs["model"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(
                serving_model_dir, "movie-recommender"
            ),
        )
    )
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 26
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [19]:
try:
    loaded = tf.saved_model.load(pusher.outputs['pushed_model'].get()[0].uri)
    scores, titles = loaded(["42"])
    print(f"Recommendations: {titles[0][:3]}")
except BaseException as err:
    logging.error(err)

ERROR:absl:Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * ['42']
    * None
    * False
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * {'title_xf': TensorSpec(shape=(None, 1), dtype=tf.string, name='title_xf'),
 'userId_xf': TensorSpec(shape=(None, 1), dtype=tf.int64, name='userId_xf')}
    * None
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * {'title_xf': TensorSpec(shape=(None, 1), dtype=tf.string, name='queries/title_xf'),
 'userId_xf': TensorSpec(shape=(None, 1), dtype=tf.int64, name='queries/userId_xf')}
    * None
    * False
  Keyword arguments: {}

Option 3:
  Positional arguments (3 total):
    * {'title_xf': TensorSpec(shape=(None, 1), dtype=tf.string, name='queries/title_xf'),
 'userId_xf': TensorSpec(shape=(None, 1), dtype=tf.int64, name='queries/userId_xf')}
    